In [1]:
# install need libraly and packages
!pip3 install pandas


  Using cached https://files.pythonhosted.org/packages/bb/71/8f53bdbcbc67c912b888b40def255767e475402e9df64050019149b1a943/pandas-1.0.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e7/f9/f0b53f88060247251bf481fa6ea62cd0d25bf1b11a87888e53ce5b7c8ad2/pytz-2019.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/07/08/a549ba8b061005bb629b76adc000f3caaaf881028b963c2e18f811c6edc1/numpy-1.18.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/65/eb/1f97cb97bfc2390a276969c6fae16075da282f5058082d4cb10c6c5c1dba/six-1.14.0-py2.py3-none-any.whl


In [15]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
# задать папку с файлами для обработки
path_ab_test = 'ab_test.tsv'
path_ab_test_res = 'ab_test.txt'

In [4]:
# read file a/b test result
df_ab_ctr = pd.read_csv(path_ab_test
                     ,encoding='cp1251'
                     , sep='\t'
                    ).fillna('') # замена пустых значений


df_ab_ctr.sample(2)

,search_id,search_date,clicks,group
3436,8370335410408280452,2017-11-23,0,2
9070,-7022929548771241561,2017-11-23,0,3


In [5]:
df_ab_ctr.describe()

,search_id,clicks,group
count,9.662000e+03,9662.000000,9662.000000
mean,5.127190e+16,0.534568,2.503001
std,5.327594e+18,0.909413,0.500017
min,-9.220836e+18,0.000000,2.000000
25%,-4.578733e+18,0.000000,2.000000
50%,1.348137e+17,0.000000,3.000000
75%,4.576849e+18,1.000000,3.000000
max,9.220794e+18,15.000000,3.000000


In [37]:
# преобразование даты в UNIX формат
df_ab_ctr['search_date'] = pd.to_datetime(df_ab_ctr['search_date'])

df_ab_ctr.head()

,search_id,search_date,clicks,group
0,-3136706724385003188,2017-11-23,0,2
1,-3139609516900744687,2017-11-22,0,2
2,-776473245209166820,2017-11-22,0,2
3,2254608150971607405,2017-11-25,1,2
4,2397168305001145852,2017-11-24,0,2


In [34]:
# определяем тествовый период
df_ab_ctr['search_date'].describe()

count                    9662
unique                      4
top       2017-11-23 00:00:00
freq                     3455
first     2017-11-22 00:00:00
last      2017-11-25 00:00:00
Name: search_date, dtype: object

In [48]:
# просчет суммы кликов в разбивке по дню и тест.группе
df_clicks = pd.pivot_table(df_ab_ctr, 
                   index=['search_date','group'],
                   values=['clicks'],
                   aggfunc=np.sum)
df_clicks

clicks
search_date group        
2017-11-22  2         939
            3         937
2017-11-23  2         980
            3         884
2017-11-24  2         573
            3         535
2017-11-25  2         183
            3         134

In [49]:
# просчет к-ства показов в разбивке по дню и тест.группе
df_view = pd.pivot_table(df_ab_ctr, 
                     index=['search_date','group'], 
                     values='search_id',
                     aggfunc=len)
df_view

search_id
search_date group           
2017-11-22  2           1663
            3           1706
2017-11-23  2           1755
            3           1700
2017-11-24  2           1084
            3           1143
2017-11-25  2            300
            3            311

In [50]:
# объединение данные по кликам и показам
pivot = pd.concat([df_clicks,df_view],axis=1)
pivot

clicks  search_id
search_date group                   
2017-11-22  2         939       1663
            3         937       1706
2017-11-23  2         980       1755
            3         884       1700
2017-11-24  2         573       1084
            3         535       1143
2017-11-25  2         183        300
            3         134        311

In [51]:
pivot.rename(columns={'search_id': 'cnt_view', 'clicks': 'sum_clicks'}, inplace=True)
pivot

sum_clicks  cnt_view
search_date group                      
2017-11-22  2             939      1663
            3             937      1706
2017-11-23  2             980      1755
            3             884      1700
2017-11-24  2             573      1084
            3             535      1143
2017-11-25  2             183       300
            3             134       311

In [55]:
pivot['CTR'] = round(pivot['sum_clicks'].div(pivot['cnt_view']) * 100, 2)
pivot

sum_clicks  cnt_view    CTR
search_date group                             
2017-11-22  2             939      1663  56.46
            3             937      1706  54.92
2017-11-23  2             980      1755  55.84
            3             884      1700  52.00
2017-11-24  2             573      1084  52.86
            3             535      1143  46.81
2017-11-25  2             183       300  61.00
            3             134       311  43.09

In [63]:
import scipy.stats  as stats

In [65]:
stats.pearsonr(pivot['sum_clicks'], pivot['cnt_view'])

(0.9930724337258161, 8.268426761843281e-07)